In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, assemble, transpile, execute
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from math import sqrt, pi

In [26]:
def cadd(ctrl, reg_a, reg_b, qc, n):
    reg_a = QuantumRegister(n+2, name="a")
    reg_b = QuantumRegister(n, name="b")
    ctrl = QuantumRegister(1, name="ctrl")
    cl = ClassicalRegister(n+1, name="cl")


    qc = QuantumCircuit(ctrl, reg_a, reg_b, cl, name="qc")

    a, b = a[::-1], b[::-1]
    #basis encoding
    for i in range(n):
        if a[i] == '1':
            qc.x(reg_a[i])
        if b[i] == '1':
            qc.x(reg_b[i])

    qc.x(ctrl[0]) #set ctrl to 1 in order to get the sum

    #step 1
    for i in range(1, n):
        qc.cx(reg_a[i], reg_b[i])


    #step 2
        #substep 1
    qc.ccx(ctrl[0], reg_a[n-1], reg_a[n])

        #substep 2
    for i in range(n-2, 0, -1):
        qc.cx(reg_a[i], reg_a[i+1])


    #step 3
    for i in range(n-1):
        qc.ccx(reg_b[i], reg_a[i], reg_a[i+1])


    #step 4
        #substep 1
    qc.ccx(reg_b[n-1], reg_a[n-1], reg_a[n+1])
        #substep 2
    qc.ccx(ctrl[0], reg_a[n+1], reg_a[n])
        #substep 3
    qc.ccx(reg_b[n-1], reg_a[n-1], reg_a[n+1])
        #substep 4
    qc.ccx(ctrl[0], reg_a[n-1], reg_b[n-1])


    #step 5
    for i in range(n-2, -1, -1):
        #substep 1
        qc.ccx(reg_b[i], reg_a[i], reg_a[i+1])
        #substep 2
        qc.ccx(ctrl[0], reg_a[i], reg_b[i])

    

    #step 6
    for i in range(1, n-1):
        qc.cx(reg_a[i], reg_a[i+1])


    #step 7
    for i in range(1, n):
        qc.cx(reg_a[i], reg_b[i])


def modified_cadd(ctrl, reg_a, reg_b, j, qc, n):
    #step 1
    for i in range(1, n):
        qc.cx(reg_a[i], reg_b[i+j])


    #step 2 #CCX needs to be changed (different Z)
    #substep 1
    qc.ccx(ctrl[j], reg_a[n-1], reg_b[n+j])

    #substep 2
    for i in range(n-2, 0, -1):
        qc.cx(reg_a[i], reg_a[i+1])


    #step 3
    for i in range(n-1):
        qc.ccx(reg_b[i+j], reg_a[i], reg_a[i+1])


    #step 4 #CCX needs to be changed (different Z)
    #substep 1
    qc.ccx(reg_b[n-1+j], reg_a[n-1], reg_b[n+1+j])
    #substep 2
    qc.ccx(ctrl[j], reg_b[n+1+j], reg_b[n+j])
    #substep 3
    qc.ccx(reg_b[n-1+j], reg_a[n-1], reg_b[n+1+j])
    #substep 4
    qc.ccx(ctrl[j], reg_a[n-1], reg_b[n-1+j])


    #step 5
    for i in range(n-2, -1, -1):
        #substep 1
        qc.ccx(reg_b[i+j], reg_a[i], reg_a[i+1])
        #substep 2
        qc.ccx(ctrl[j], reg_a[i], reg_b[i+j])

    

    #step 6
    for i in range(1, n-1):
        qc.cx(reg_a[i], reg_a[i+1])


    #step 7
    for i in range(1, n):
        qc.cx(reg_a[i], reg_b[i+j])


def multiply(a, b, n):
    reg_a = QuantumRegister(n, name="a")
    reg_b = QuantumRegister(n, name="b")
    reg_p = QuantumRegister((2*n)+1, name="p")
    cl = ClassicalRegister((2*n)+1, name="cl")
    qc = QuantumCircuit(reg_a, reg_b, reg_p, cl, name="qc")
    
    a, b = a[2:], b[2:]
    a, b = a.zfill(n), b.zfill(n)
    a, b = a[::-1], b[::-1]
    
    #basis encoding
    for i in range(n):
        if a[i] == '1':
            qc.x(reg_a[i])
        if b[i] == '1':
            qc.x(reg_b[i])
    
    qc.i(reg_p[j] for j in range(2*n+1))
    #step 1
    for i in range(n):
        qc.ccx(reg_b[0], reg_a[i], reg_p[i])
    
    qc.barrier()
    
    #step 2
    modified_cadd(reg_b, reg_a, reg_p, 1, qc, n)
    qc.barrier()
    for k in range(n-1):
        for j in range(2, n):
            modified_cadd(reg_b, reg_a, reg_p, j, qc, n)
        qc.barrier()    
     
    #step 3
    for i in range(2*n):
        qc.measure(reg_p[i], i)
    
    return qc
    
    
    

a = bin(int(input()))
b = bin(int(input()))

n = 5
qc = multiply(a, b, n)
sim = Aer.get_backend('aer_simulator')
result = sim.run(assemble(qc)).result().get_counts()
print(result)
# qc.draw(output = 'mpl')

2
3
{'00000000110': 1024}


### 4